# Find Patterns Forming Clumps in a String

[ba1e](https://rosalind.info/problems/ba1e/)

Given integers L and t, a string Pattern forms an (L, t)-clump inside a (larger) string Genome if there is an interval of Genome of length L in which Pattern appears at least t times. For example, TGCA forms a (25,3)-clump in the following Genome: gatcagcataagggtcccTGCAATGCATGACAAGCCTGCAgttgttttac.

### Clump Finding Problem

Find patterns forming clumps in a string.

    Given: 

A string Genome, and integers k, L, and t.

    Return: 

All distinct k-mers forming (L, t)-clumps in Genome.

Sample Dataset

    CGGACTCGACAGATGTGAAGAAATGTGAAGACTGAGTGAAGAGAAGAGGAAACACGACACGACATTGCGACATAATGTACGAATGTAATGTGCCTATGGC
    5 75 4

Sample Output

    CGACA GAAGA AATGT

In [48]:
from collections import defaultdict

In [49]:
def get_frequent_words(seq, k, t):
    clumps = set()
    kmers = defaultdict(int)
    for i in range(len(seq) - k + 1):
        kmers[seq[i:i+k]] += 1
        if kmers[seq[i:i+k]] >= t:
            clumps.add(seq[i:i+k])
    
    return clumps

In [50]:
def get_clumps(dna, k, L, t):
    clumps = set()
    for i in range(len(dna) - L + 1):
        clumps.update(get_frequent_words(dna[i:i+L], k, t))
    return clumps

In [51]:
file = "rosalind_ba1e.txt" 
# file = "input.txt" 
with open(file, 'r') as f:
    lines = f.readlines()
    dna  = lines[0].strip()
    k, L, t = map(int, lines[1].split())

print(' '.join(get_clumps(dna, k, L, t)))

CTTCAAGCGT CTATTATTGC GAACGCCAGG GAGTTACATG TAACAACCTC CCGGTACTGT
